In [1]:
import face_recognition
import os
import numpy as np


def get_gropped_face(image_file):
    image = face_recognition.load_image_file(image_file)
    face_locations = face_recognition.face_locations(image)
    
    
    a, b, c, d = face_locations[0]
    cropped_face = image[a:c,d:b,:]
  
    
    return cropped_face

def get_face_embedding(face):
    return face_recognition.face_encodings(face)


def get_face_embedding_dict(dir_path):
    file_list=os.listdir(dir_path)
    embedding_dict = {}
    
    for file in file_list:
        img_path = os.path.join(dir_path, file)

        try:
            face = get_gropped_face(img_path)
            embedding = get_face_embedding(face)
            if len(embedding) > 0:  
                embedding_dict[os.path.splitext(file)[0]]= embedding[0]
        except:
            continue
       
# 얼굴영역 face가 제대로 detect되지 않으면  len(embedding)==0인 경우가 발생하므로 
# os.path.splitext(file)[0]에는 이미지파일명에서 확장자를 제거한 이름이 담깁니다. 


    return embedding_dict

def get_distance(name1, name2):
    return np.linalg.norm(embedding_dict[name1]-embedding_dict[name2], ord=2)



# name1과 name2의 거리를 비교하는 함수를 생성하되, name1은 미리 지정하고, name2는 호출시에 인자로 받도록 합니다.
def get_sort_key_func(name1):
    def get_distance_from_name1(name2):
        return get_distance(name1, name2)
    return get_distance_from_name1

  
# 이렇게 생성된 함수 sort_key_func는 sort_key_func('obama') 라고 호출할 때 trump와 obama 사이의 임베딩 벡터 거리를 계산합니다.



dir_path = os.getenv('HOME')+'/aiffel/face_embedding/actor'
embedding_dict = get_face_embedding_dict(dir_path)




sort_key_func = get_sort_key_func('유퉁') 


#(sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))


def get_nearest_face(name, top=10):#len(embedding_dict)-1):
    sort_key_func = get_sort_key_func(name)
    sorted_faces = sorted(embedding_dict.items(), key=lambda x:sort_key_func(x[0]))

    for i in range(top+1):
        if i == 0 :   # 첫번째로 나오는 이름은 자기 자신일 것이므로 제외합시다. 
            continue
        if sorted_faces[i]:
            print('순위 {} : 이름({}), 거리({})'.format(i, sorted_faces[i][0], sort_key_func(sorted_faces[i][0])))

            
get_nearest_face('현석종_1')


순위 1 : 이름(현석종_2), 거리(0.387998635083565)
순위 2 : 이름(임지규), 거리(0.3949137374694232)
순위 3 : 이름(김진엽), 거리(0.4116087931211047)
순위 4 : 이름(안도규), 거리(0.424993047474134)
순위 5 : 이름(이태성), 거리(0.42513072047212036)
순위 6 : 이름(우도환), 거리(0.4257750520720103)
순위 7 : 이름(남궁민), 거리(0.4260198303769896)
순위 8 : 이름(윤현민), 거리(0.43623643014695873)
순위 9 : 이름(김수현), 거리(0.4373511608845674)
순위 10 : 이름(윤균상), 거리(0.4427298904238862)
